In [1]:
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras import *
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import *

import random

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Y_train = y_train
Y_test = y_test


num_classes = 10 
x_train = x_train/255.
y_train = utils.to_categorical(y_train, num_classes) 

x_test = x_test/255.
y_test = utils.to_categorical(y_test, num_classes) 

In [3]:
##########################################Functions to rename vgg19 model ###################################

def add_infix(a,b,character):  
    b.startswith(a)
    tail = b.split(a, 1)[1]
    b = a + character + tail
    return b

add_infix('block1_conv1','block1_conv1_ib-0','A')

def rename_network_nodes(vgg19,character):  # https://stackoverflow.com/questions/63373692/how-to-rename-the-layers-of-a-keras-model-without-corrupting-the-structure
    layer_names = {layer._name for layer in vgg19.layers}
    _network_nodes = []
    for b in vgg19._network_nodes:
        for a in layer_names:
            if(b.startswith(a)):
                _network_nodes.append(add_infix(a,b,character))
    vgg19._network_nodes = set(_network_nodes)
    
def rename(vgg19,character):        
    rename_network_nodes(vgg19,character)
    
    vgg19._name = vgg19._name + f"{character}"
    for layer in vgg19.layers:
        layer._name = layer._name + f"{character}"

######################################################################################################

In [4]:
def build_model(weight_decay= 0.0005, x_shape=[32,32,3], num_classes=10):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

    model = Sequential()
#     model.add(Input(shape=()))

    model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes))
#     model.add(Activation('softmax'))
    return model

In [5]:
########################################## Building siamese network ###################################


def build_siamese_vgg_model(shape, class_num):
    inputs1 = Input(shape)
    inputs2 = Input(shape)
    

    x1 = build_model()
    x2 = build_model()

    x = layers.concatenate([x1.outputs[0], x2.outputs[0]])
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)  

    # build the model
    model = Model([x1.inputs[0], x2.inputs[0]], outputs)
    
    model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

model = build_siamese_vgg_model(x_train[0].shape, num_classes)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_13_input (InputLayer)    [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1792        conv2d_input[0][0]               
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 32, 32, 64)   1792        conv2d_13_input[0][0]            
______________________________________________________________________________________________

In [6]:
# x2.summary()

In [7]:
# x1 = build_model()
# # print(x1.summary())
# x2 = build_model()
# # print(x2.summary())

# x = layers.concatenate([x1.outputs[0], x2.outputs[0]])

In [8]:
def make_pairs(images, labels):
    pairImages = []
    pairLabels = []

    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]

    for idxA in range(len(images)):
        currentImage = images[idxA]
        label = labels[idxA]
#         print(currentImage.shape, label)

        
        for idxB in random.sample(list(idx[label[0]]), 1):
            posImage = images[idxB]
#             print(posImage.shape)
#             print('\n\n\n')


            pairImages.append(np.array([currentImage, posImage]))
            pairLabels.append(np.array(label))
        
    return np.array(pairImages), np.array(pairLabels)

In [9]:
######################################################################################################

x_train_siamese, y_train_siamese = make_pairs(x_train, Y_train)
x_test_siamese, y_test_siamese = make_pairs(x_test, Y_test)

y_train_siamese = utils.to_categorical(y_train_siamese, num_classes) 
y_test_siamese = utils.to_categorical(y_test_siamese, num_classes) 

x_train_both = [x_train_siamese[:,0,:], x_train_siamese[:,1,:]]
x_test_both = [x_test_siamese[:,0,:], x_test_siamese[:,1,:]]


print('Images')
print(np.array(x_train_both).shape)
print(np.array(x_test_both).shape)


print('Labels')
print(y_train_siamese.shape)
print(y_test_siamese.shape)

Images
(2, 50000, 32, 32, 3)
(2, 10000, 32, 32, 3)
Labels
(50000, 10)
(10000, 10)


In [10]:


# print(model.summary())

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)

history3 = model.fit(x_train_both, y_train_siamese, epochs=400, batch_size=512, shuffle=True,
         validation_data=(x_test_both, y_test_siamese),callbacks=[callback])


print('Saving model')
model.save('model.h5')

Epoch 1/50
98/98 [==============================] - 28s 171ms/step - loss: 7.1824 - accuracy: 0.1279 - val_loss: 6.7765 - val_accuracy: 0.1000
Epoch 2/50
98/98 [==============================] - 14s 138ms/step - loss: 6.6557 - accuracy: 0.2191 - val_loss: 7.2342 - val_accuracy: 0.1001
Epoch 3/50
98/98 [==============================] - 14s 138ms/step - loss: 6.3062 - accuracy: 0.2929 - val_loss: 8.3008 - val_accuracy: 0.1004
Epoch 4/50
98/98 [==============================] - 13s 138ms/step - loss: 6.1198 - accuracy: 0.3440 - val_loss: 9.0661 - val_accuracy: 0.1004
Epoch 5/50
98/98 [==============================] - 14s 138ms/step - loss: 5.9979 - accuracy: 0.3736 - val_loss: 8.5711 - val_accuracy: 0.1141
Epoch 6/50
98/98 [==============================] - 13s 138ms/step - loss: 5.8953 - accuracy: 0.4051 - val_loss: 7.6755 - val_accuracy: 0.1524
Epoch 7/50
98/98 [==============================] - 13s 138ms/step - loss: 5.8158 - accuracy: 0.4294 - val_loss: 8.2753 - val_accuracy: 0.1385

NameError: name 'model' is not defined